## Get the combined Dataframes from Jan-17 to Jun-17 for Pickups originating at LGA = 138

In [1]:
from tqdm import tqdm
import glob
import pandas as pd

In [2]:
csvFiles = glob.glob("../raw_data/taxidata/*.csv")

In [3]:
combined_df = []

for file in tqdm(csvFiles):
    df = pd.read_csv(file, index_col = None, header = 0)
    df = df.loc[df["PULocationID"]==138, :]
    df = df.reset_index()
    df = df.drop("index", axis=1)
    combined_df.append(df)

100%|██████████| 6/6 [03:26<00:00, 34.39s/it]


In [4]:
final_df = pd.concat(combined_df, ignore_index=True)

In [5]:
final_df.shape

(1471188, 17)

In [ ]:
final_df.to_csv('../clean_data/TaxiData_Jan17-Jun17.gz', compression='gzip', index=False)